In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt

import remap_values as rv
import good_plots   as gp

import cPickle

In [2]:
train_df = pd.read_csv('inp_data/train.csv')
test_df  = pd.read_csv('inp_data/test.csv')

train_df, test_df = rv.run_tan( train_df, test_df )

remap_values.py:554: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  inds = df[ df['PoolQC'].isnull() ][ df['PoolArea'] > 0 ].index


In [3]:
big_df = train_df.drop( 'SalePrice', axis=1).append( test_df, ignore_index=True)

In [4]:
num_df = train_df.ix[:, train_df.dtypes==float ].copy()
num_df = num_df.join( train_df.ix[:, train_df.dtypes==int ], how='inner'  ).copy()

In [5]:
cat_df = train_df.ix[:, train_df.dtypes=='O'].copy()

In [6]:
qual_cols = ['ExterQual', 'ExterCond', 'GarageQual', 'GarageCond', 'FireplaceQu', 'KitchenQual', 'HeatingQC', 'BsmtQual']
qual_map  = {'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}

for col in qual_cols:
    num_df[col] = cat_df[col].replace( qual_map )

In [7]:
expo_map = {'None': 0, 'No': 1, 'Mn': 2, 'Av': 3, 'Gd': 4}
num_df['BsmtExposure'] = cat_df['BsmtExposure'].replace( expo_map )

In [9]:
bfin_map = {'None': 1, 'Unf': 1, 'LwQ': 2,'Rec': 3, 'BLQ': 4, 'ALQ': 5, 'GLQ': 6}
num_df['BsmtFinType1'] = cat_df['BsmtFinType1'].replace( bfin_map )
num_df['BsmtFinType2'] = cat_df['BsmtFinType2'].replace( bfin_map )

In [10]:
func_map = {'None': 0, 'Sal': 1, 'Sev': 2, 'Maj2': 3, 'Maj1': 4, 'Mod': 5, 'Min2': 6, 'Min1': 7, 'Typ': 8}
num_df['Functional'] = cat_df['Functional'].replace( func_map )

In [11]:
gfin_map = {'None': 1,'Unf': 1, 'RFn': 1, 'Fin': 2}
num_df['GarageFinish'] = cat_df['GarageFinish'].replace( gfin_map )

In [12]:
fence_map = {'None': 1, 'MnWw': 1, 'GdWo': 1, 'MnPrv': 2, 'GdPrv': 4}
num_df['Fence'] = cat_df['Fence'].replace( fence_map )

In [68]:
MSd_map = {'20': 1, '30': 0, '40': 0, '45': 0,'50': 0, '60': 1, '70': 0, '75': 0, '80': 0, 
           '85': 0, '90': 0, '120': 1, '150': 0, '160': 0, '180': 0, '190': 0}

num_df['NewerDwelling'] = cat_df['MSZoning'].replace( {'RL':0, 'RM':1, 'C (all)':2, 'FV':3, 'RH':4} )

In [18]:
manual_cols = ['BsmtExposure','BsmtFinType1','BsmtFinType2','Functional','GarageFinish','Fence','NewerDwelling']

In [15]:
binarize = ['LotShape','LandContour','LandSlope','Electrical','GarageType','PavedDrive','MiscFeature']

In [14]:
# Alter pretty binary categories so just regular/irregular, 0 reg 1 irreg
num_df['LotShape']    = (cat_df['LotShape'   ] == 'Reg'   ) * 1
num_df['LandContour'] = (cat_df['LandContour'] == 'Lvl'   ) * 1
num_df['LandSlope'  ] = (cat_df['LandSlope'  ] == 'Gtl'   ) * 1
num_df['Electrical' ] = (cat_df['Electrical' ] == 'SBrkr' ) * 1
num_df['GarageType' ] = (cat_df['GarageType' ] == 'Detchd') * 1
num_df['PavedDrive' ] = (cat_df['PavedDrive' ] == 'Y'     ) * 1
num_df['MiscFeature'] = (cat_df['MiscFeature'] == 'Shed'  ) * 1

In [16]:
num_df['HasWoodDeck']   = (num_df['WoodDeckSF'  ] > 0) * 1
num_df['Has2ndFlr']     = (num_df['2ndFlrSF'    ] > 0) * 1
num_df['HasMasVnr']     = (num_df['MasVnrArea'  ] > 0) * 1
num_df['Remodeled']     = (num_df['YearBuilt'   ] != num_df['YearRemodAdd']) * 1
num_df['RecentRemodel'] = (num_df['YearRemodAdd'] >= num_df['YrSold'      ]) * 1
num_df['NewHouse']      = (num_df['YearBuilt'   ] == num_df['YrSold'      ]) * 1

In [17]:
binarize2 = ['2ndFlrSF', 'MasVnrArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch']

for col in binarize2:
  num_df['Has'+col] = (num_df[col] != 0) * 1

In [29]:
num_df['HighSeason'] = ((num_df['MoSold'] >= 5) & (num_df['MoSold'] <= 7) ) * 1

In [32]:
neigh_map = {'MeadowV': 0, 'IDOTRR' : 1, 'Sawyer' : 1, 'BrDale' : 1, 'OldTown': 1, 'Edwards': 1, 
             'BrkSide': 1, 'Blueste': 1, 'SWISU'  : 2, 'NAmes'  : 2, 'NPkVill': 2, 'Mitchel': 2,
             'SawyerW': 2, 'Gilbert': 2, 'NWAmes' : 2, 'Blmngtn': 2, 'CollgCr': 2, 'ClearCr': 3, 
             'Crawfor': 3, 'Veenker': 3, 'Somerst': 3, 'Timber' : 3, 'StoneBr': 4, 'NoRidge': 4, 
             'NridgHt': 4}
num_df['NeighBin'] = cat_df['Neighborhood'].replace( neigh_map )

In [33]:
num_df['PartialPlan'] = (cat_df['SaleCondition'] == 'Partial') * 1

In [34]:
heat_map = {'Po': 0, 'Fa': 1, 'TA': 2, 'Gd': 3, 'Ex': 4}

num_df['HeatingScale'] = cat_df['HeatingQC'].replace( heat_map )

In [35]:
num_df['AreaInside'] = num_df['1stFlrSF'] + num_df['2ndFlrSF']

In [36]:
num_df['Age']              =             2010 - num_df['YearBuilt']
num_df['TimeSinceSold']    =             2010 - num_df['YrSold']
num_df['YearSinceRemodel'] = num_df['YrSold'] - num_df['YearRemodAdd']

In [40]:
#
# ONLY TRAIN
#
num_df = num_df[ num_df['GrLivArea'] < 4000 ]

In [41]:
#PCA?
#Preprocess?

In [72]:
col_list = num_df.columns
print num_df.columns

Index([u'LotFrontage', u'MasVnrArea', u'GarageYrBlt', u'Id', u'MSSubClass',
       u'LotArea', u'OverallQual', u'OverallCond', u'YearBuilt',
       u'YearRemodAdd', u'BsmtFinSF1', u'BsmtFinSF2', u'BsmtUnfSF',
       u'TotalBsmtSF', u'1stFlrSF', u'2ndFlrSF', u'LowQualFinSF', u'GrLivArea',
       u'BsmtFullBath', u'BsmtHalfBath', u'FullBath', u'HalfBath',
       u'BedroomAbvGr', u'KitchenAbvGr', u'TotRmsAbvGrd', u'Fireplaces',
       u'GarageCars', u'GarageArea', u'WoodDeckSF', u'OpenPorchSF',
       u'EnclosedPorch', u'3SsnPorch', u'ScreenPorch', u'PoolArea', u'MiscVal',
       u'MoSold', u'YrSold', u'SalePrice', u'ExterQual', u'ExterCond',
       u'GarageQual', u'GarageCond', u'FireplaceQu', u'KitchenQual',
       u'HeatingQC', u'BsmtQual', u'BsmtExposure', u'BsmtFinType1',
       u'BsmtFinType2', u'Functional', u'GarageFinish', u'Fence',
       u'NewerDwelling', u'LotShape', u'LandContour', u'LandSlope',
       u'Electrical', u'GarageType', u'PavedDrive', u'MiscFeature',
       u'HasW

In [71]:
corr_series = num_df.corrwith( num_df['SalePrice'] ).sort_values( ascending=False )

In [86]:
# Already binary
binary_class = ['GarageFinish','LotShape','LandContour','LandSlope','Electrical','GarageType','PavedDrive',
                'MiscFeature','HasWoodDeck','Has2ndFlr','HasMasVnr','Remodeled','RecentRemodel','NewHouse',
                'Has2ndFlrSF','HasMasVnrArea','HasWoodDeckSF','HasOpenPorchSF','HasEnclosedPorch',
                'Has3SsnPorch','HasScreenPorch','HighSeason','PartialPlan']

In [91]:
# Need to make binary
to_bin_class = ['BsmtExposure','BsmtFinType1','BsmtFinType2','Functional','Fence','NewerDwelling','NeighBin','HeatingScale','MSSubClass']

In [90]:
num_df['NBath'] = num_df['BsmtFullBath'] + num_df['BsmtHalfBath']*0.5 + num_df['FullBath'] + num_df['HalfBath']*0.5
num_df = num_df.drop( ['BsmtFullBath','BsmtHalfBath','FullBath','HalfBath'], axis=1 )